In [1]:
import scipy.io
import jax.numpy as np
from scipy.io import loadmat 
import numpy as onp
import nibabel as nib
import matplotlib.pyplot as plt 
import os
import shutil
from PIL import Image
from inrmri.bart import bart_acquisition_from_arrays
from inrmri.data_harvard import get_csmaps_and_mask, get_reference_reco
import pandas as pd
from jax import jit, vmap, random
from inrmri.dip import TimeDependant_DIP_Net, helix_generator, circle_generator
from inrmri.new_radon import ForwardRadonOperator
from inrmri.fourier import fastshiftfourier, get_freqs
from inrmri.basic_nn import weighted_loss 
from inrmri.utils import to_complex, is_inside_of_radial_lim, meshgrid_from_subdiv_autolims, total_variation_batch_complex, save_matrix_and_dict_in_zpy, load_matrix_and_dict_from_zpy    
from inrmri.utils import create_exp_file_name, total_variation_complex
import optax 
from inrmri.metrics_rd import mean_psnr, mean_ssim, mean_artifact_power
from inrmri.utils_rdls import seconds_to_min_sec_format, filter_and_get_columns, apply_transform

from inrmri.image_processor import BeforeLinRegNormalizer
from inrmri.basic_plotting import full_halph_FOV_space_time 
from inrmri.image_processor import reduce_FOV 

from inrmri.utils_rdls import safe_normalize, get_center

from inrmri.utils_rdls import get_info_volunteer, read_segmentation, read_ref_dataset

## General

In [2]:
total_slices          = 8
num_frames            = 30
base_path             = '/mnt/workspace/datasets/pulseqCINE/'
target_columns        = ['training_name', 'psnr', 'ssim', 'it', 'duration [min]', 'duration [s]']

## Volunteer

In [3]:
dataset        = 'DATA_0.55T'
volunteer      = 'FB'
# --- PATH ---
base_folder                     = base_path + dataset + '/' + volunteer + '/'
train_data_folder               = base_folder + 'traindata/'
segmentation_folder             = base_folder + 'segmentations/'
segmentation_endo_folder        = segmentation_folder + 'endocardium/'
segmentation_septum_folder      = segmentation_folder + 'septum/'

In [4]:
es_folder = base_folder + 'es/'
if not os.path.exists(es_folder):
    os.makedirs(es_folder)

## DIP Model reconstructions

In [5]:
stdip_parameters = {
    'experiment_name': 'iter_lr_init_value', 
    'training_params': {
        'iter':2000,
        'lr_init_value': 1e-3,
        'slice':None,
    }
}

tddip_parameters = {
    'experiment_name': 'iter_lr_init_value', 
    'training_params': {
        'iter':2000,
        'lr_init_value': 1e-3,
        'slice':None,
    }
}

In [6]:
stdip_parameters['path']               = base_folder + 'stDIP'
stdip_parameters['csv_path']           = stdip_parameters['path'] +  '/' + stdip_parameters['experiment_name'] + ".csv"
stdip_parameters['csv_path_summary']   = stdip_parameters['path'] +  '/' + stdip_parameters['experiment_name'] + "_summary.csv"

In [7]:
tddip_parameters['path']               = base_folder + 'tdDIP'
tddip_parameters['csv_path']           = tddip_parameters['path'] +  '/' + tddip_parameters['experiment_name'] + ".csv"
tddip_parameters['csv_path_summary']   = tddip_parameters['path'] +  '/' + tddip_parameters['experiment_name'] + "_summary.csv"

## Data

In [8]:
import os

all_slices_data = []

for i in range(1, total_slices + 1):
    print(f'slice {i}')
    slice_data = {}

    # === Volunteer and Training Parameters ===
    volunteer_params = get_info_volunteer(dataset, volunteer, i)
    stdip_parameters['training_params']['slice'] = i
    tddip_parameters['training_params']['slice'] = i

    # === Reference Dataset Path ===
    dataset_name = f'slice_{i}_{total_slices}_nbins{num_frames}'
    path_dataset = os.path.join(train_data_folder, f'{dataset_name}.npz')

    # === Reference Reconstructions ===
    recon_fs, recon_grasp, recon_sense, time_grasp, time_sense = read_ref_dataset(path_dataset)
    recon_grasp    = get_center(recon_grasp) 
    recon_sense    = get_center(recon_sense)
    recon_fs       = get_center(recon_fs) 
    recon_grasp    = safe_normalize(recon_grasp) 
    recon_sense    = safe_normalize(recon_sense) 
    recon_fs       = safe_normalize(recon_fs) 
    recon_fs = apply_transform(recon_fs, volunteer_params['trans_gt'])
    recon_grasp = apply_transform(recon_grasp, volunteer_params['trans'])
    recon_sense = apply_transform(recon_sense, volunteer_params['trans'])
    slice_data['recon'] = {
        'fs': recon_fs,
        'grasp': recon_grasp,
        'sense': recon_sense
    }

    # === stDIP Reconstruction ===
    stdip_parameters['exp_folder_path'] = os.path.join(stdip_parameters['path'], dataset_name, stdip_parameters['experiment_name'])
    df_stdip = pd.read_csv(stdip_parameters['csv_path'], delimiter=';')
    stDIP_results = filter_and_get_columns(df_stdip, stdip_parameters['training_params'], target_columns)[0]
    stdip_parameters['best_recon_path'] = os.path.join(stdip_parameters['exp_folder_path'], stDIP_results['training_name'], 'best_recon.npz')
    recon_stdip = onp.load(stdip_parameters['best_recon_path'], allow_pickle=True)['best_recon']
    recon_stdip = safe_normalize(recon_stdip)
    recon_stdip = apply_transform(recon_stdip, volunteer_params['trans'])
    slice_data['recon']['stdip'] = recon_stdip

    # === tdDIP Reconstruction ===
    tddip_parameters['exp_folder_path'] = os.path.join(tddip_parameters['path'], dataset_name, tddip_parameters['experiment_name'])
    df_tddip = pd.read_csv(tddip_parameters['csv_path'], delimiter=';')
    tdDIP_results = filter_and_get_columns(df_tddip, tddip_parameters['training_params'], target_columns)[0]
    tddip_parameters['best_recon_path'] = os.path.join(tddip_parameters['exp_folder_path'], tdDIP_results['training_name'], 'best_recon.npz')
    recon_tddip = onp.load(tddip_parameters['best_recon_path'], allow_pickle=True)['best_recon']
    recon_stdip = safe_normalize(recon_tddip)
    recon_tddip = apply_transform(recon_tddip, volunteer_params['trans'])
    slice_data['recon']['tddip'] = recon_tddip

    # === fs Segmentations ===
    seg_endo, seg_endo_fill, seg_end_area = read_segmentation(
        os.path.join(segmentation_endo_folder, f"{volunteer}_recon_fs_slice{i}.nii"))
    seg_sep = read_segmentation(
        os.path.join(segmentation_septum_folder, f"{volunteer}_recon_fs_slice{i}.nii"), fill=False)
    slice_data['segmentation'] = {
        'fs': {
            'sep': seg_sep,
            'endo': seg_endo,
            'endo_fill': seg_endo_fill,
            'area_ed': seg_end_area[volunteer_params['EF_frames']['EDV_gt']],
            'area_es': seg_end_area[volunteer_params['EF_frames']['ESV_gt']]
        }
    }

    # === GRASP Segmentations ===
    seg_endo, seg_endo_fill, seg_end_area = read_segmentation(
        os.path.join(segmentation_endo_folder, f"{volunteer}_grasp_slice{i}.nii"))
    seg_sep = read_segmentation(
        os.path.join(segmentation_septum_folder, f"{volunteer}_grasp_slice{i}.nii"), fill=False)
    slice_data['segmentation']['grasp'] = {
        'sep': seg_sep,
        'endo': seg_endo,
        'endo_fill': seg_endo_fill,
        'area_ed': seg_end_area[volunteer_params['EF_frames']['EDV']],
        'area_es': seg_end_area[volunteer_params['EF_frames']['ESV']]
    }
    
    # === stDIP Segmentations ===
    seg_endo, seg_endo_fill, seg_end_area = read_segmentation(
        os.path.join(segmentation_endo_folder, f"{volunteer}_stdip_slice{i}.nii"))
    seg_sep = read_segmentation(
        os.path.join(segmentation_septum_folder, f"{volunteer}_stdip_slice{i}.nii"), fill=False)
    slice_data['segmentation']['stdip'] = {
        'sep': seg_sep,
        'endo': seg_endo,
        'endo_fill': seg_endo_fill,
        'area_ed': seg_end_area[volunteer_params['EF_frames']['EDV']],
        'area_es': seg_end_area[volunteer_params['EF_frames']['ESV']]
    }

    # === tdDIP Segmentations ===
    seg_endo, seg_endo_fill, seg_end_area = read_segmentation(
        os.path.join(segmentation_endo_folder, f"{volunteer}_tddip_slice{i}.nii"))
    seg_sep = read_segmentation(
        os.path.join(segmentation_septum_folder, f"{volunteer}_tddip_slice{i}.nii"), fill=False)
    slice_data['segmentation']['tddip'] = {
        'sep': seg_sep,
        'endo': seg_endo,
        'endo_fill': seg_endo_fill,
        'area_ed': seg_end_area[volunteer_params['EF_frames']['EDV']],
        'area_es': seg_end_area[volunteer_params['EF_frames']['ESV']]
    }

    # === Save Slice Info ===
    slice_data['metadata'] = {
        'slice_number': i,
        'dataset_name': dataset_name,
        'volunteer_params': volunteer_params
    }

    all_slices_data.append(slice_data)


slice 1


2025-08-04 11:27:38.461974: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version (12.6.20). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


FileNotFoundError: No such file or no access: '/mnt/workspace/datasets/pulseqCINE/DATA_0.55T/FB/segmentations/septum/FB_grasp_slice1.nii'

In [ ]:
all_slices_data[0]['metadata']['volunteer_params']

## Functions

In [ ]:
from scipy.ndimage import center_of_mass

def get_center_of_mass(segmentation_2d):
    cy, cx = center_of_mass(segmentation_2d)
    return cy, cx

def get_center_of_mass_dict(segmentation_3d):
    centers_dict = {
        'cx': [], 
        'cy': []
    }
    for i in range(segmentation_3d.shape[-1]):
        cy, cx = get_center_of_mass(segmentation_3d[..., i])
        centers_dict['cy'].append(  cy  )
        centers_dict['cx'].append(  cx  )
    return centers_dict
    
def get_random_point(segmentation):
    """Returns a random (row, col) point from a 2D binary segmentation."""
    indices = onp.argwhere(segmentation > 0)
    if indices.size == 0:
        raise ValueError("Segmentation is empty.")
    random_index = onp.random.choice(len(indices))
    return tuple(indices[random_index])

def get_random_point_list(segmentation_3d):
    random_dict = {
        'cx': [], 
        'cy': []
    }
    for i in range(segmentation_3d.shape[-1]):
        cy, cx = get_random_point(segmentation_3d[..., i])
        random_dict['cy'].append(  cy  )
        random_dict['cx'].append(  cx  )
    return random_dict

In [ ]:
def get_all_points(segmentation):
    """Returns a list of (row, col) points from a 2D binary segmentation."""
    indices = onp.argwhere(segmentation > 0)
    if indices.size == 0:
        raise ValueError("Segmentation is empty.")
    return [tuple(idx) for idx in indices]

def get_all_points_per_slice(segmentation_3d):
    """
    Returns a dictionary with lists of (cy, cx) points for each slice 
    in a 3D binary segmentation.
    """
    all_points_dict = {
        'cx': [],  # List of lists
        'cy': []   # Each sublist contains all cx/cy for a slice
    }

    for i in range(segmentation_3d.shape[-1]):
        indices = onp.argwhere(segmentation_3d[..., i] > 0)
        if indices.size == 0:
            all_points_dict['cy'].append([])
            all_points_dict['cx'].append([])
        else:
            cy_list = [int(pt[0]) for pt in indices]
            cx_list = [int(pt[1]) for pt in indices]
            all_points_dict['cy'].append(cy_list)
            all_points_dict['cx'].append(cx_list)

    return all_points_dict

In [ ]:
from inrmri.image_processor import crop_2d_im 
from inrmri.sharpness import (
    radial_sharpness,
    extract_line_at_points,
    get_line_points_from_angle,
    find_percentile_idx, 
    get_distance
)

In [ ]:
import numpy as np
from scipy.ndimage import map_coordinates

def get_line_points(p1, p2, num=100):
    """
    Get equally spaced points between p1 and p2.
    Args:
        p1: (x1, y1)
        p2: (x2, y2)
        num: number of points
    Returns:
        2 x N array with row coords and col coords
    """
    x_vals = np.linspace(p1[0], p2[0], num)
    y_vals = np.linspace(p1[1], p2[1], num)
    return np.vstack([x_vals, y_vals])

def sample_line_from_image(image, p1, p2, num=100):
    """
    Sample image intensity along a line between two points.
    Args:
        image: 2D numpy array
        p1, p2: (x, y) start and end coordinates (row, col)
        num: number of points along the line
    Returns:
        values: intensity values along the line
        coords: (2 x N) coordinates sampled
    """
    coords = get_line_points(p1, p2, num)
    values = map_coordinates(image, coords, order=1, mode='reflect')
    return values, coords

def plot_image_and_projection(image, coords, values, interpolation_factor, vmax=1.0, sigmoid_fit=False):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    x_values = [i / interpolation_factor for i in range(len(values))]

    # Image with line
    ax1 = axes[0]
    ax1.imshow(np.clip(image, 0, vmax), cmap='bone', vmin=0, vmax=vmax)
    ax1.scatter(coords[1], coords[0], s=1, c='red')  # coords: (row, col)
    ax1.set_title("Image and Sampled Line")
    ax1.axis('off')

    # Projection
    ax2 = axes[1]
    ax2.plot(x_values, values, 'b.', label='Profile')
    
    if sigmoid_fit:
        fitted, params, x_centered, r2 = fit_sigmoid(values)
        if fitted is not None:
            ax2.plot(x_values, fitted, 'r-', label=f'Fitted Sigmoid\nR²={r2:.3f}')
        else:
            ax2.set_title("Sigmoid fit failed")
    else:
        ax2.plot(x_values,values, 'b.')

    ax2.set_title("Intensity Profile Along Line")
    ax2.set_xlabel("Sample Index")
    ax2.set_ylabel("Intensity")
    ax2.legend()

    plt.tight_layout()
    plt.show()
    if sigmoid_fit:
        return fitted, params, x_centered, r2

In [ ]:
import math
def compute_distance_two_points_2d(point1,point2, bool_round=False):
    distance = math.sqrt((point2[0] - point1[0])**2 + (point2[1] - point1[1])**2)
    if bool_round:
        distance = round(distance)
    return distance

def first_index_greater_than(lst, value):
    for i, v in enumerate(lst):
        if v > value:
            return i
    return -1  # or None if no such element exists

In [ ]:
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

def safe_sigmoid(x, L, x0, k, b):
    z = -k * (x - x0)
    z = np.clip(z, -500, 500)
    return L / (1 + np.exp(z)) + b

def fit_sigmoid(profile, p0=None, bounds=None, show_plot=False):
    y = np.asarray(profile)
    estimated_center_value = (np.max(y) + np.min(y))/2
    estimated_center_idx = first_index_greater_than(y, estimated_center_value)
    x_centered = np.arange(len(y)) - estimated_center_idx

    if p0 is None:
        p0 = [np.max(y) - np.min(y), 0, 0.1, np.min(y)]
    if bounds is None:
        bounds = ([0, -np.inf, -10, -np.inf], [np.inf, np.inf, 10, np.inf])

    try:
        popt, _ = curve_fit(
            safe_sigmoid, x_centered, y, p0,
            method='trf', maxfev=10000, bounds=bounds
        )
        fitted_profile = safe_sigmoid(x_centered, *popt)
        r2 = r2_score(y, fitted_profile)

        if show_plot:
            plt.figure()
            plt.plot(x_centered, y, 'bo', label='Data')
            plt.plot(x_centered, fitted_profile, 'r-', label='Fit')
            plt.legend()
            plt.title(f"Sigmoid Fit (R² = {r2:.3f})")
            plt.xlabel('Centered x')
            plt.ylabel('y')
            plt.grid(True)
            plt.show()

        return fitted_profile, popt, x_centered, r2
    except RuntimeError:
        print("Sigmoid fit did not converge.")
        return None, None, x_centered, None



In [ ]:
def compute_20_80_statistics(values, coords):
    # find_percentile_idx funciona en un vector ordenado y entrega el indice más
    # cercano al valor min + (max-min)*p/100, con p=20 y 80
    idx_sorting   = np.argsort(values)
    idx_sorted_20, p20 = find_percentile_idx(values[idx_sorting], 20) 
    idx_sorted_80, p80 = find_percentile_idx(values[idx_sorting], 80) 
    
    # encontrar los índeces del vector original 
    corrected_idxs = np.arange(values.shape[0])[idx_sorting]
    idx_20 = corrected_idxs[idx_sorted_20]
    idx_80 = corrected_idxs[idx_sorted_80]
    
    # get_distance(points) es 1 si los puntos están en horizontal o vertical 
    # pero es mayor cuando los puntos están en diagonal
    d_20_80 = np.abs(idx_20 - idx_80)
    d       = d_20_80 * get_distance(coords.T)
    dict_20_80_statistics = {
        'p20':p20,
        'p80':p80,
        'idx_20':idx_20,
        'idx_80':idx_80,
        'd_20_80':d_20_80,
        'd':d,
    }
    return dict_20_80_statistics

def plot_20_80_statistics(values, dict_20_80_statistics, interpolation_factor):
    p20     = dict_20_80_statistics['p20'] 
    p80     = dict_20_80_statistics['p80']
    idx_20  = dict_20_80_statistics['idx_20']   / interpolation_factor
    idx_80  = dict_20_80_statistics['idx_80']   / interpolation_factor
    d_20_80 = dict_20_80_statistics['d_20_80']  / interpolation_factor
    d       = dict_20_80_statistics['d']

    x_values = [i / interpolation_factor for i in range(len(values))]
    
    plt.plot(x_values, values, '.-', color = 'red', label= 'profile')
    
    p20_color = 'orange'
    plt.hlines([p20], xmin=0, xmax=14, color=p20_color, label='p20', linestyles=(0,(1,3)))
    plt.vlines([idx_20], ymin=values.min(), ymax=values.max(), color=p20_color, label='idx_20', linestyles=(0,(1,1)))
    
    p80_color = 'blue'
    plt.hlines([p80], xmin=0, xmax=14, color=p80_color, label='p80', linestyles=(0, (3,3,1,3)))
    plt.vlines([idx_80], ymin=values.min(), ymax=values.max(), color=p80_color, label='idx_80', linestyles=(0, (3,1,1,1)))
    
    plt.hlines([values.min()], xmin=idx_20, xmax=idx_80, color='gray', linestyles='-')
    plt.text(x=(idx_20 + idx_80)/2, y=values.min() + 0.07 * (values.max() - values.min()), s='$d_{20, 80} = $' + f'{d_20_80}', ha='center', va='center', color='gray')
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.title('$d_{20,80} = ' f'{d:.3f}$, sharpness 1/d {1 / d:.3f}')
    plt.tight_layout()
    plt.show()

# 1 .- Select points

In [ ]:
interpolation_factor = 100
selected_slice = 3

In [ ]:
slice_data       = all_slices_data[selected_slice]
recon            = np.abs(slice_data['recon']['fs'])
seg_endo         = slice_data['segmentation']['fs']['endo']
seg_sep          = slice_data['segmentation']['fs']['sep']
volunteer_params = all_slices_data[selected_slice]['metadata']['volunteer_params']
slice_idx_EDV    = volunteer_params['EF_frames']['EDV']
slice_idx_ESV    = volunteer_params['EF_frames']['ESV']
centers_dict = get_center_of_mass_dict(seg_endo)
random_dict  = get_random_point_list(seg_sep)
all_dict     = get_all_points_per_slice(seg_sep)
vmax         = volunteer_params['saturation']

## 1.1 .- EDV

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns

# --- Left subplot: Endocardium + center ---
ax = axes[0]
ax.imshow(np.clip(recon[:, :, slice_idx_EDV], 0, vmax), cmap='bone', vmin=0, vmax=vmax)
ax.contour(seg_endo[:, :, slice_idx_EDV], levels=[0.5], colors='red', linewidths=1.5)
ax.plot(centers_dict['cx'][slice_idx_EDV], centers_dict['cy'][slice_idx_EDV], 'yo', markersize=6)
ax.set_title('Endocardium + Center')

# --- Right subplot: Septum + random point ---
ax = axes[1]
ax.imshow(np.clip(recon[:, :, slice_idx_EDV], 0, vmax), cmap='bone', vmin=0, vmax=vmax)
ax.contour(seg_sep[:, :, slice_idx_EDV], levels=[0.5], colors='lime', linewidths=1.5)
ax.plot(random_dict['cx'][slice_idx_EDV], random_dict['cy'][slice_idx_EDV], 'bo', markersize=6)
ax.set_title('Septum + Random Point')

plt.tight_layout()
plt.show()

In [ ]:
p2 = (centers_dict['cy'][slice_idx_EDV], centers_dict['cx'][slice_idx_EDV])
for i in range(  len(all_dict['cx'][0])  ):
    p1 = (all_dict['cy'][slice_idx_EDV][i], all_dict['cx'][slice_idx_EDV][i])
    values, coords = sample_line_from_image(recon[:,:,slice_idx_EDV], p1, p2, num=interpolation_factor*compute_distance_two_points_2d(p1, p2, bool_round=True))
    plot_image_and_projection(recon[:,:,slice_idx_EDV], coords, values, interpolation_factor, vmax=vmax, sigmoid_fit = True)

In [ ]:
p2 = (centers_dict['cy'][slice_idx_EDV], centers_dict['cx'][slice_idx_EDV])
p1 = (all_dict['cy'][slice_idx_EDV][3], all_dict['cx'][slice_idx_EDV][3])
values, coords = sample_line_from_image(recon[:,:,slice_idx_EDV], p1, p2, num=interpolation_factor*compute_distance_two_points_2d(p1, p2, bool_round=True))
fitted, params, x_centered, r2 = plot_image_and_projection(recon[:,:,slice_idx_EDV], coords, values, interpolation_factor, vmax=vmax, sigmoid_fit = True)

In [ ]:
np.max(values), np.min(values), np.max(values) - np.min(values)

In [ ]:
plt.plot(x_centered, values)

## 1.2 .- ESV

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 5))  # 1 row, 2 columns

# --- Left subplot: Endocardium + center ---
ax = axes[0]
ax.imshow(np.clip(recon[:, :, slice_idx_ESV], 0, vmax), cmap='bone', vmin=0, vmax=vmax)
ax.contour(seg_endo[:, :, slice_idx_ESV], levels=[0.5], colors='red', linewidths=1.5)
ax.plot(centers_dict['cx'][slice_idx_ESV], centers_dict['cy'][slice_idx_ESV], 'yo', markersize=6)
ax.set_title('Endocardium + Center')

# --- Right subplot: Septum + random point ---
ax = axes[1]
ax.imshow(np.clip(recon[:, :, slice_idx_ESV], 0, vmax), cmap='bone', vmin=0, vmax=vmax)
ax.contour(seg_sep[:, :, slice_idx_ESV], levels=[0.5], colors='lime', linewidths=1.5)
ax.plot(random_dict['cx'][slice_idx_ESV], random_dict['cy'][slice_idx_ESV], 'bo', markersize=6)
ax.set_title('Septum + Random Point')

plt.tight_layout()
plt.show()

In [ ]:
p2 = (centers_dict['cy'][slice_idx_ESV], centers_dict['cx'][slice_idx_ESV])
for i in range(  len(all_dict['cx'][slice_idx_ESV])  ):
    p1 = (all_dict['cy'][slice_idx_ESV][i], all_dict['cx'][slice_idx_ESV][i])
    values, coords = sample_line_from_image(recon[:,:,slice_idx_ESV], p1, p2, num=interpolation_factor*compute_distance_two_points_2d(p1, p2, bool_round=True))
    plot_image_and_projection(recon[:,:,slice_idx_ESV], coords, values, interpolation_factor, vmax=vmax, sigmoid_fit = True)

# 4 .- Get 20 80 statistics per line

In [ ]:
p2 = (centers_dict['cy'][0], centers_dict['cx'][0])
for i in range(  len(all_dict['cx'][0])  ):
    p1 = (all_dict['cy'][0][i], all_dict['cx'][0][i])
    values, coords = sample_line_from_image(recon[:,:,0], p1, p2, num=interpolation_factor*compute_distance_two_points_2d(p1, p2, bool_round=True))
    fitted, params, x_centered, r2 = fit_sigmoid(values)
    dict_20_80_statistics =  compute_20_80_statistics(fitted, coords)
    plot_20_80_statistics(fitted, dict_20_80_statistics, interpolation_factor)

## Create a function that gets the mean edge sharpness

In [ ]:
slice_idx_EDV = volunteer_params['EF_frames']['EDV']
slice_idx_ESV = volunteer_params['EF_frames']['ESV']

In [ ]:
def get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx):
    centers_dict = get_center_of_mass_dict(seg_endo)
    all_dict     = get_all_points_per_slice(seg_sep)
    # data from the selected frame
    recon_slice = recon[:,:,slice_idx]
    # loop
    d_list = []
    p2 = (centers_dict['cy'][slice_idx], centers_dict['cx'][slice_idx])
    for i in range(  len(all_dict['cx'][slice_idx])  ):
        p1 = (all_dict['cy'][slice_idx][i],  all_dict['cx'][slice_idx][i])
        values, coords = sample_line_from_image(recon_slice, p1, p2, num=round(interpolation_factor*compute_distance_two_points_2d(p1, p2)))
        fitted, params, x_centered, r2 = fit_sigmoid(values)
        dict_20_80_statistics =  compute_20_80_statistics(fitted, coords)
        d_list.append(dict_20_80_statistics['d'])
    return d_list

# FS

In [ ]:
edv_all = []
esv_all = []

es_edv_all = []
es_esv_all = []

edv_mean = []
esv_mean = []
edv_std = []
esv_std = []

es_edv_mean = []
es_esv_mean = []
es_edv_std = []
es_esv_std = []

for selected_slice in range(8):
    print('slice: ', str(selected_slice))
    # slice data
    slice_data       = all_slices_data[selected_slice]
    recon            = np.abs(slice_data['recon']['fs'])
    seg_endo         = slice_data['segmentation']['fs']['endo']
    seg_sep          = slice_data['segmentation']['fs']['sep']
    volunteer_params = all_slices_data[selected_slice]['metadata']['volunteer_params']
    slice_idx_EDV    = volunteer_params['EF_frames']['EDV_gt']
    slice_idx_ESV    = volunteer_params['EF_frames']['ESV_gt']
    # get edge sharpness
    d_list_edv = get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx_EDV)
    d_list_esv = get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx_ESV)
    edv_all = edv_all + d_list_edv
    esv_all = esv_all + d_list_esv
    d_list_edv = np.array(d_list_edv)
    d_list_esv = np.array(d_list_esv)
    es_edv_all = es_edv_all + list(1 / d_list_edv)
    es_esv_all = es_edv_all + list(1 / d_list_esv)
    # save
    edv_mean.append(np.mean(d_list_edv))
    esv_mean.append(np.mean(d_list_esv))
    edv_std.append(np.std(d_list_edv))
    esv_std.append(np.std(d_list_esv))

    es_edv_mean.append(np.mean(1 / d_list_edv))
    es_esv_mean.append(np.mean(1 / d_list_esv))
    es_edv_std.append(np.std(1 / d_list_edv))
    es_esv_std.append(np.std(1 / d_list_esv))

In [ ]:
plt.plot(edv_mean)
plt.plot(esv_mean)

In [ ]:
plt.plot(es_edv_mean)
plt.plot(es_esv_mean)

In [ ]:
print('overall distance EDV: ' , round(np.mean(edv_all),2) ,  ' +- ', round(np.std(edv_all),2))
print('overall distance ESV: ' , round(np.mean(esv_all),2) ,  ' +- ', round(np.std(esv_all),2))

In [ ]:
print('overall edge sharpness EDV: ' , round(np.mean(es_edv_all),2) ,  ' +- ', round(np.std(es_edv_all),2))
print('overall edge sharpness ESV: ' , round(np.mean(es_esv_all),2) ,  ' +- ', round(np.std(es_esv_all),2))

In [ ]:
print('overall distance EDV: ' , round(np.mean(edv_mean),2) ,  ' +- ', round(np.std(edv_mean),2))
print('overall distance ESV: ' , round(np.mean(esv_mean),2) ,  ' +- ', round(np.std(esv_mean),2))

In [ ]:
print('overall edge sharpness EDV: ' , round(np.mean(es_edv_mean),2) ,  ' +- ', round(np.std(es_edv_mean),2))
print('overall edge sharpness ESV: ' , round(np.mean(es_esv_mean),2) ,  ' +- ', round(np.std(es_esv_mean),2))

# ST-DIP

In [ ]:
edv_all = []
esv_all = []

es_edv_all = []
es_esv_all = []

edv_mean = []
esv_mean = []
edv_std = []
esv_std = []

es_edv_mean = []
es_esv_mean = []
es_edv_std = []
es_esv_std = []

for selected_slice in range(8):
    print('slice: ', str(selected_slice))
    # slice data
    slice_data       = all_slices_data[selected_slice]
    recon            = np.abs(slice_data['recon']['stdip'])
    seg_endo         = slice_data['segmentation']['stdip']['endo']
    seg_sep          = slice_data['segmentation']['stdip']['sep']
    volunteer_params = all_slices_data[selected_slice]['metadata']['volunteer_params']
    slice_idx_EDV    = volunteer_params['EF_frames']['EDV']
    slice_idx_ESV    = volunteer_params['EF_frames']['ESV']
    # get edge sharpness
    d_list_edv = get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx_EDV)
    d_list_esv = get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx_ESV)
    edv_all = edv_all + d_list_edv
    esv_all = esv_all + d_list_esv
    d_list_edv = np.array(d_list_edv)
    d_list_esv = np.array(d_list_esv)
    es_edv_all = es_edv_all + list(1 / d_list_edv)
    es_esv_all = es_edv_all + list(1 / d_list_esv)
    # save
    edv_mean.append(np.mean(d_list_edv))
    esv_mean.append(np.mean(d_list_esv))
    edv_std.append(np.std(d_list_edv))
    esv_std.append(np.std(d_list_esv))

    es_edv_mean.append(np.mean(1 / d_list_edv))
    es_esv_mean.append(np.mean(1 / d_list_esv))
    es_edv_std.append(np.std(1 / d_list_edv))
    es_esv_std.append(np.std(1 / d_list_esv))

In [ ]:
plt.plot(edv_mean)
plt.plot(esv_mean)

In [ ]:
plt.plot(es_edv_mean)
plt.plot(es_esv_mean)

In [ ]:
print('overall distance EDV: ' , round(np.mean(edv_all),2) ,  ' +- ', round(np.std(edv_all),2))
print('overall distance ESV: ' , round(np.mean(esv_all),2) ,  ' +- ', round(np.std(esv_all),2))

In [ ]:
print('overall edge sharpness EDV: ' , round(np.mean(es_edv_all),2) ,  ' +- ', round(np.std(es_edv_all),2))
print('overall edge sharpness ESV: ' , round(np.mean(es_esv_all),2) ,  ' +- ', round(np.std(es_esv_all),2))

In [ ]:
print('overall distance EDV: ' , round(np.mean(edv_mean),2) ,  ' +- ', round(np.std(edv_mean),2))
print('overall distance ESV: ' , round(np.mean(esv_mean),2) ,  ' +- ', round(np.std(esv_mean),2))

In [ ]:
print('overall edge sharpness EDV: ' , round(np.mean(es_edv_mean),2) ,  ' +- ', round(np.std(es_edv_mean),2))
print('overall edge sharpness ESV: ' , round(np.mean(es_esv_mean),2) ,  ' +- ', round(np.std(es_esv_mean),2))

# TD-DIP

In [ ]:
edv_all = []
esv_all = []

es_edv_all = []
es_esv_all = []

edv_mean = []
esv_mean = []
edv_std = []
esv_std = []

es_edv_mean = []
es_esv_mean = []
es_edv_std = []
es_esv_std = []

for selected_slice in range(8):
    print('slice: ', str(selected_slice))
    # slice data
    slice_data       = all_slices_data[selected_slice]
    recon            = np.abs(slice_data['recon']['tddip'])
    seg_endo         = slice_data['segmentation']['stdip']['endo']
    seg_sep          = slice_data['segmentation']['stdip']['sep']
    volunteer_params = all_slices_data[selected_slice]['metadata']['volunteer_params']
    slice_idx_EDV    = volunteer_params['EF_frames']['EDV']
    slice_idx_ESV    = volunteer_params['EF_frames']['ESV']
    # get edge sharpness
    d_list_edv = get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx_EDV)
    d_list_esv = get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx_ESV)
    edv_all = edv_all + d_list_edv
    esv_all = esv_all + d_list_esv
    d_list_edv = np.array(d_list_edv)
    d_list_esv = np.array(d_list_esv)
    es_edv_all = es_edv_all + list(1 / d_list_edv)
    es_esv_all = es_edv_all + list(1 / d_list_esv)
    # save
    edv_mean.append(np.mean(d_list_edv))
    esv_mean.append(np.mean(d_list_esv))
    edv_std.append(np.std(d_list_edv))
    esv_std.append(np.std(d_list_esv))

    es_edv_mean.append(np.mean(1 / d_list_edv))
    es_esv_mean.append(np.mean(1 / d_list_esv))
    es_edv_std.append(np.std(1 / d_list_edv))
    es_esv_std.append(np.std(1 / d_list_esv))

In [ ]:
plt.plot(edv_mean)
plt.plot(esv_mean)

In [ ]:
plt.plot(es_edv_mean)
plt.plot(es_esv_mean)

In [ ]:
print('overall distance EDV: ' , round(np.mean(edv_all),2) ,  ' +- ', round(np.std(edv_all),2))
print('overall distance ESV: ' , round(np.mean(esv_all),2) ,  ' +- ', round(np.std(esv_all),2))

In [ ]:
print('overall edge sharpness EDV: ' , round(np.mean(es_edv_all),2) ,  ' +- ', round(np.std(es_edv_all),2))
print('overall edge sharpness ESV: ' , round(np.mean(es_esv_all),2) ,  ' +- ', round(np.std(es_esv_all),2))

In [ ]:
print('overall distance EDV: ' , round(np.mean(edv_mean),2) ,  ' +- ', round(np.std(edv_mean),2))
print('overall distance ESV: ' , round(np.mean(esv_mean),2) ,  ' +- ', round(np.std(esv_mean),2))

In [ ]:
print('overall edge sharpness EDV: ' , round(np.mean(es_edv_mean),2) ,  ' +- ', round(np.std(es_edv_mean),2))
print('overall edge sharpness ESV: ' , round(np.mean(es_esv_mean),2) ,  ' +- ', round(np.std(es_esv_mean),2))

# GRASP

In [ ]:
edv_all = []
esv_all = []

es_edv_all = []
es_esv_all = []

edv_mean = []
esv_mean = []
edv_std = []
esv_std = []

es_edv_mean = []
es_esv_mean = []
es_edv_std = []
es_esv_std = []

for selected_slice in range(8):
    print('slice: ', str(selected_slice))
    # slice data
    slice_data       = all_slices_data[selected_slice]
    recon            = np.abs(slice_data['recon']['grasp'])
    seg_endo         = slice_data['segmentation']['stdip']['endo']
    seg_sep          = slice_data['segmentation']['stdip']['sep']
    volunteer_params = all_slices_data[selected_slice]['metadata']['volunteer_params']
    slice_idx_EDV    = volunteer_params['EF_frames']['EDV']
    slice_idx_ESV    = volunteer_params['EF_frames']['ESV']
    # get edge sharpness
    d_list_edv = get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx_EDV)
    d_list_esv = get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx_ESV)
    edv_all = edv_all + d_list_edv
    esv_all = esv_all + d_list_esv
    d_list_edv = np.array(d_list_edv)
    d_list_esv = np.array(d_list_esv)
    es_edv_all = es_edv_all + list(1 / d_list_edv)
    es_esv_all = es_edv_all + list(1 / d_list_esv)
    # save
    edv_mean.append(np.mean(d_list_edv))
    esv_mean.append(np.mean(d_list_esv))
    edv_std.append(np.std(d_list_edv))
    esv_std.append(np.std(d_list_esv))

    es_edv_mean.append(np.mean(1 / d_list_edv))
    es_esv_mean.append(np.mean(1 / d_list_esv))
    es_edv_std.append(np.std(1 / d_list_edv))
    es_esv_std.append(np.std(1 / d_list_esv))

In [ ]:
plt.plot(edv_mean)
plt.plot(esv_mean)

In [ ]:
plt.plot(es_edv_mean)
plt.plot(es_esv_mean)

In [ ]:
print('overall distance EDV: ' , round(np.mean(edv_all),2) ,  ' +- ', round(np.std(edv_all),2))
print('overall distance ESV: ' , round(np.mean(esv_all),2) ,  ' +- ', round(np.std(esv_all),2))

In [ ]:
print('overall edge sharpness EDV: ' , round(np.mean(es_edv_all),2) ,  ' +- ', round(np.std(es_edv_all),2))
print('overall edge sharpness ESV: ' , round(np.mean(es_esv_all),2) ,  ' +- ', round(np.std(es_esv_all),2))

In [ ]:
print('overall distance EDV: ' , round(np.mean(edv_mean),2) ,  ' +- ', round(np.std(edv_mean),2))
print('overall distance ESV: ' , round(np.mean(esv_mean),2) ,  ' +- ', round(np.std(esv_mean),2))

In [ ]:
print('overall edge sharpness EDV: ' , round(np.mean(es_edv_mean),2) ,  ' +- ', round(np.std(es_edv_mean),2))
print('overall edge sharpness ESV: ' , round(np.mean(es_esv_mean),2) ,  ' +- ', round(np.std(es_esv_mean),2))

# SENSE

In [ ]:
edv_all = []
esv_all = []

es_edv_all = []
es_esv_all = []

edv_mean = []
esv_mean = []
edv_std = []
esv_std = []

es_edv_mean = []
es_esv_mean = []
es_edv_std = []
es_esv_std = []

for selected_slice in range(8):
    print('slice: ', str(selected_slice))
    # slice data
    slice_data       = all_slices_data[selected_slice]
    recon            = np.abs(slice_data['recon']['sense'])
    seg_endo         = slice_data['segmentation']['stdip']['endo']
    seg_sep          = slice_data['segmentation']['stdip']['sep']
    volunteer_params = all_slices_data[selected_slice]['metadata']['volunteer_params']
    slice_idx_EDV    = volunteer_params['EF_frames']['EDV']
    slice_idx_ESV    = volunteer_params['EF_frames']['ESV']
    # get edge sharpness
    d_list_edv = get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx_EDV)
    d_list_esv = get_edge_sharpness(recon, seg_endo, seg_sep, interpolation_factor, slice_idx_ESV)
    edv_all = edv_all + d_list_edv
    esv_all = esv_all + d_list_esv
    d_list_edv = np.array(d_list_edv)
    d_list_esv = np.array(d_list_esv)
    es_edv_all = es_edv_all + list(1 / d_list_edv)
    es_esv_all = es_edv_all + list(1 / d_list_esv)
    # save
    edv_mean.append(np.mean(d_list_edv))
    esv_mean.append(np.mean(d_list_esv))
    edv_std.append(np.std(d_list_edv))
    esv_std.append(np.std(d_list_esv))

    es_edv_mean.append(np.mean(1 / d_list_edv))
    es_esv_mean.append(np.mean(1 / d_list_esv))
    es_edv_std.append(np.std(1 / d_list_edv))
    es_esv_std.append(np.std(1 / d_list_esv))

In [ ]:
slice_idx_EDV, slice_idx_ESV

In [ ]:
plt.plot(edv_mean)
plt.plot(esv_mean)

In [ ]:
plt.plot(es_edv_mean)
plt.plot(es_esv_mean)

In [ ]:
print('overall distance EDV: ' , round(np.mean(edv_all),2) ,  ' +- ', round(np.std(edv_all),2))
print('overall distance ESV: ' , round(np.mean(esv_all),2) ,  ' +- ', round(np.std(esv_all),2))

In [ ]:
print('overall edge sharpness EDV: ' , round(np.mean(es_edv_all),2) ,  ' +- ', round(np.std(es_edv_all),2))
print('overall edge sharpness ESV: ' , round(np.mean(es_esv_all),2) ,  ' +- ', round(np.std(es_esv_all),2))

In [ ]:
print('overall distance EDV: ' , round(np.mean(edv_mean),2) ,  ' +- ', round(np.std(edv_mean),2))
print('overall distance ESV: ' , round(np.mean(esv_mean),2) ,  ' +- ', round(np.std(esv_mean),2))

In [ ]:
print('overall edge sharpness EDV: ' , round(np.mean(es_edv_mean),2) ,  ' +- ', round(np.std(es_edv_mean),2))
print('overall edge sharpness ESV: ' , round(np.mean(es_esv_mean),2) ,  ' +- ', round(np.std(es_esv_mean),2))